In [1]:
!pip install -q huggingface_hub[hf_xet]
!pip install -q -U wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 80.5 MB/s eta 0:00:00


In [2]:
import wandb
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")

login(token = hf_token)

wb_token = user_secrets.get_secret("WANDB_API_KEY")
wandb.login(key=wb_token)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: urakiny (causal_language_trainer) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
from datasets import load_dataset

daa_file_path='/kaggle/input/building-the-fine-tuning-dataset/train.csv'

train_ds=load_dataset('csv', data_files=daa_file_path)['train']
train_ds

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['anchor', 'positive'],
    num_rows: 4984
})

In [4]:
train_ds[0]

{'anchor': '<think>\nLet’s think through this step by step\nApril: 48 clips\nMay: 48 / 2 = 24 clips\nTotal: 48 + 24 = 72 clips\n</think>\n\\boxed{72}',
 'positive': '<think>\nLet’s think through this step by step\nApril: 48 clips\nMay: 48 / 2 = 24 clips\nTotal = 48 + 24 = 72 clips\n</think>\n\\boxed{72}'}

# Pre-processing dataset

We will split dataset into training, testing and evaluation.

In [5]:
train_test_split=train_ds.train_test_split(test_size=0.1, seed=42)
train_test_split

DatasetDict({
    train: Dataset({
        features: ['anchor', 'positive'],
        num_rows: 4485
    })
    test: Dataset({
        features: ['anchor', 'positive'],
        num_rows: 499
    })
})

In [6]:
train_eval_split=train_test_split['train'].train_test_split(test_size=0.1, seed=42)
train_eval_split

DatasetDict({
    train: Dataset({
        features: ['anchor', 'positive'],
        num_rows: 4036
    })
    test: Dataset({
        features: ['anchor', 'positive'],
        num_rows: 449
    })
})

In [7]:
from datasets import DatasetDict

findal_ds=DatasetDict({
    'train': train_eval_split['train'],
    'test': train_eval_split['test'],
    'eval': train_test_split['test']
})
findal_ds

DatasetDict({
    train: Dataset({
        features: ['anchor', 'positive'],
        num_rows: 4036
    })
    test: Dataset({
        features: ['anchor', 'positive'],
        num_rows: 449
    })
    eval: Dataset({
        features: ['anchor', 'positive'],
        num_rows: 499
    })
})

In [8]:
MODEL_NAME='all-MiniLM-L6-v2'
OUTPUT_DIR='encoder-L6-V2/training'
MODEL_DIR='aisuko/encoder-L6-V2'

# Load the Model

In [9]:
# https://www.sbert.net/docs/package_reference/sentence_transformer/losses.html#multiplenegativesrankingloss
import torch
from sentence_transformers import SentenceTransformer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model=SentenceTransformer(MODEL_NAME).to(device)
model

2025-07-31 01:25:43.262557: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753925143.438704      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753925143.486775      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [10]:
from sentence_transformers.losses import MultipleNegativesRankingLoss

loss=MultipleNegativesRankingLoss(model)

In [11]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SimilarityFunction

eval_dataset = load_dataset("sentence-transformers/stsb", split="validation")

dev_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=eval_dataset["sentence1"],
    sentences2=eval_dataset["sentence2"],
    scores=eval_dataset["score"],
    main_similarity=SimilarityFunction.COSINE,
    name="sts-dev",
)

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/471k [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/142k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/108k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

In [12]:
from sentence_transformers import SentenceTransformerTrainingArguments
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers import SentenceTransformerTrainer
from transformers import EarlyStoppingCallback


train_args=SentenceTransformerTrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=20,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=1e-4,
    warmup_ratio=0.1,
    fp16=True,
    bf16=False,
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # losses that use "in-batch negatives" benefit from no duplicates
    eval_strategy='steps',
    eval_steps=100,
    save_strategy='steps',
    save_steps=100,
    save_total_limit=2,
    logging_steps=100,
    report_to='wandb',
    run_name='encoder-L6-V2'
)

# evaluate the base model
dev_evaluator(model)

trainer=SentenceTransformerTrainer(
    model=model,
    args=train_args,
    train_dataset=findal_ds['train'],
    eval_dataset=findal_ds['test'],
    loss=loss,
    evaluator=dev_evaluator,
    
)

# early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=3)

# trainer.train(callbacks=[early_stopping_callback])

trainer.train()

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

wandb: WARNING Changes to your `wandb` environment variables will be ignored because your `wandb` session has already started. For more information on how to modify your settings with `wandb.init()` arguments, please refer to https://wandb.me/wandb-init.
wandb: Tracking run with wandb version 0.21.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250731_012608-41mubqwt
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run encoder-L6-V2
wandb: ⭐️ View project at https://wandb.ai/causal_language_trainer/sentence-transformers
wandb: 🚀 View run at https://wandb.ai/causal_language_trainer/sentence-transformers/runs/41mubqwt


Step,Training Loss,Validation Loss,Sts-dev Pearson Cosine,Sts-dev Spearman Cosine
100,0.021900,0.001796,0.876157,0.873649
200,0.007400,0.006022,0.876837,0.874930
300,0.003700,0.003059,0.879262,0.876646
400,0.001700,0.003603,0.879359,0.876734
500,0.007900,0.005759,0.878262,0.876599
600,0.002900,0.001324,0.878032,0.875072
700,0.002700,0.001653,0.874788,0.870834
800,0.005700,0.008796,0.875576,0.873261
900,0.003100,0.002176,0.871966,0.868923
1000,0.001500,0.001610,0.871930,0.869315


TrainOutput(global_step=5060, training_loss=0.0016430294974026061, metrics={'train_runtime': 516.9419, 'train_samples_per_second': 156.149, 'train_steps_per_second': 9.788, 'total_flos': 0.0, 'train_loss': 0.0016430294974026061, 'epoch': 20.0})

In [13]:
# evaluate the trained model on the test set
test_dataset = load_dataset("sentence-transformers/stsb", split="test")

test_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=eval_dataset["sentence1"],
    sentences2=eval_dataset["sentence2"],
    scores=eval_dataset["score"],
    main_similarity=SimilarityFunction.COSINE,
    name="sts-test",
)

test_evaluator(model)

{'sts-test_pearson_cosine': 0.8694545678565029,
 'sts-test_spearman_cosine': 0.8656345719034876}

In [14]:
model.save_pretrained(MODEL_DIR)

In [15]:
import kagglehub

VARIATION_SLUG='v0.1.4'

handler = f'aisuko/encoder-L6-V2/transformers/{VARIATION_SLUG}'

kagglehub.model_upload(
    handle=handler,
    local_model_dir=MODEL_DIR,
    license_name='MIT',
    version_notes='FT with reasoning paths'
)

Uploading Model https://www.kaggle.com/models/aisuko/encoder-L6-V2/transformers/v0.1.4 ...
Starting upload for file aisuko/encoder-L6-V2/special_tokens_map.json


Uploading: 100%|██████████| 695/695 [00:00<00:00, 1.05kB/s]

Upload successful: aisuko/encoder-L6-V2/special_tokens_map.json (695B)
Starting upload for file aisuko/encoder-L6-V2/config.json


Uploading: 100%|██████████| 617/617 [00:00<00:00, 971B/s]

Upload successful: aisuko/encoder-L6-V2/config.json (617B)
Starting upload for file aisuko/encoder-L6-V2/tokenizer.json


...

Uploading: 100%|██████████| 90.9M/90.9M [00:01<00:00, 51.8MB/s]

Upload successful: aisuko/encoder-L6-V2/model.safetensors (87MB)
Starting upload for file aisuko/encoder-L6-V2/1_Pooling/config.json


Uploading: 100%|██████████| 296/296 [00:00<00:00, 490B/s]

Upload successful: aisuko/encoder-L6-V2/1_Pooling/config.json (296B)


Your model instance version has been created.
Files are being processed...
See at: https://www.kaggle.com/models/aisuko/encoder-L6-V2/transformers/v0.1.4
